In [1]:
from web3 import Web3
import requests
import time

In [2]:
import seaborn as sn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
w3 = Web3(Web3.HTTPProvider("https://api.avax.network/ext/bc/C/rpc"))

lb_proxy = Web3.toChecksumAddress("0xee5A90098b270596Ec35D637b30d908C862c86df")
abi=requests.get(f"https://api.snowtrace.io/api?module=contract&action=getabi&address={lb_proxy}").json()['result']

In [4]:
lb_impl = Web3.toChecksumAddress("0xD446eb1660F766d533BeCeEf890Df7A69d26f7d1")
contract = w3.eth.contract(address=lb_impl, abi=abi)

In [5]:
def get_decimals(address):
    abi=requests.get(f"https://api.snowtrace.io/api?module=contract&action=getabi&address={Web3.toChecksumAddress(address)}").json()['result']

    contract = w3.eth.contract(address=address, abi=abi)
    return contract.functions.decimals().call()

In [6]:
tokenX = contract.functions.getTokenX().call()
tokenY = contract.functions.getTokenY().call()

In [7]:
active_bin = contract.functions.getActiveId().call()
left_bins = [contract.functions.getNextNonEmptyBin(True, active_bin).call()]
right_bins = [contract.functions.getNextNonEmptyBin(False, active_bin).call()]


In [8]:
while True:
    next_bin = contract.functions.getNextNonEmptyBin(True, left_bins[0]).call()
    if next_bin > left_bins[0] or len(left_bins)==250:
        break
    else:
        left_bins.insert(0, next_bin)

print("Found all USDC bins, looking for AVAX bins...")

while True:
    next_bin = contract.functions.getNextNonEmptyBin(False, right_bins[-1]).call()
    if next_bin < right_bins[-1] or len(right_bins)==250:
        break
    else:
        right_bins.append(next_bin)

Found all USDC bins, looking for AVAX bins...


In [9]:
target_bins = left_bins + [active_bin] + right_bins

In [17]:
import multiprocessing
import os

def process_bin(bin):
    bin_step = 0.002
    reserveX, reserveY = contract.functions.getBin(bin).call()
    bin_price = (1+bin_step)**(bin-2**23)
    print(bin, end="\r")
    return {"bin_id" : bin, "reserveX" : reserveX, "reserveY" : reserveY, "bin_price" : bin_price}


def get_liquidity_shape_parallel(target_bins: list) -> list:
    data = []

    with multiprocessing.Pool(processes=os.cpu_count()) as pool:
        results = pool.map(process_bin, target_bins)

    return results


In [18]:
start = int(time.time())
get_liquidity_shape_parallel(target_bins[:5])
print(f"Time taken: {int(time.time())-start} seconds")

In [ ]:
def get_liquidity_shape(contract, target_bins: list) -> list:
    data = []
    bin_step = 0.002

    print("Retreiving all bins...")

    for i, bin in enumerate(target_bins):
        print(f"Retrieving bin {i} out of {len(target_bins)}. {len(target_bins)-i} bins left", end="\r")
        reserveX, reserveY = contract.functions.getBin(bin).call()
        bin_price = (1+bin_step)**(bin-2**23)
        data.append({"bin_id" : bin, "reserveX" : reserveX, "reserveY" : reserveY, "bin_price" : bin_price})

    print("Liquidity shape retrieved")

    return data

In [29]:
data = []
bin_step = 0.002


for i, bin in enumerate(target_bins):
    print(f"Retrieving bin {i} out of {len(target_bins)}. {len(target_bins)-i} bins left", end="\r")
    reserveX, reserveY = contract.functions.getBin(bin).call()
    bin_price = (1+bin_step)**(bin-2**23)
    data.append({"bin_id" : bin, "reserveX" : reserveX, "reserveY" : reserveY, "bin_price" : bin_price})

In [30]:
df = pd.DataFrame.from_dict(data)
df.set_index('bin_id')

,reserveX,reserveY,bin_price
bin_id,,,
8375648,0,6027060,5.679747e-12
8375649,0,6027060,5.691106e-12
8375650,0,184276460,5.702488e-12
8375651,0,6027060,5.713893e-12
8375652,0,6027060,5.725321e-12
...,...,...,...
8376144,151251875747780449050,0,1.530097e-11
8376145,151955755209071448555,0,1.533157e-11
8376146,161211151528193941472,0,1.536223e-11


In [64]:
df[(df['reserveX'] == 0) & (df['reserveY'] == 0)]

,bin_id,reserveX,reserveY,bin_price,reserveX_in_Y
200,8376937,0.0,0.0,25276.689233,0.0
201,8376938,0.0,0.0,25314.604267,0.0
202,8376939,0.0,0.0,25352.576173,0.0
203,8376940,0.0,0.0,25390.605038,0.0
204,8376941,0.0,0.0,25428.690945,0.0
...,...,...,...,...,...
1220,8377957,0.0,0.0,116598.480609,0.0
1221,8377958,0.0,0.0,116773.378330,0.0
1222,8377959,0.0,0.0,116948.538398,0.0
1223,8377960,0.0,0.0,117123.961205,0.0


In [72]:
df[(df['reserveX'] > 0) & (df['reserveY'] > 0)]

,bin_id,reserveX,reserveY,bin_price,reserveX_in_Y
107,8376844,25469.22881,1026.656607,0.175121,4460.199384


In [31]:
tokenX_decimals = 18
tokenY_decimals = 6

tokenX_symbol = "AVAX"
tokenY_symbol = "USDC"

df['reserveX'] = df['reserveX'].div(10**tokenX_decimals)
df['reserveY'] = df['reserveY'].div(10**tokenY_decimals) 
df['bin_price'] = df['bin_price'] * 10**(tokenX_decimals-tokenY_decimals)
df["reserveX_in_Y"] = df['reserveX'] * df['bin_price']
df

,bin_id,reserveX,reserveY,bin_price,reserveX_in_Y
0,8375648,0.0,6.02706,5.679747,0.0
1,8375649,0.0,6.02706,5.691106,0.0
2,8375650,0.0,184.27646,5.702488,0.0
3,8375651,0.0,6.02706,5.713893,0.0
4,8375652,0.0,6.02706,5.725321,0.0
...,...,...,...,...,...
495,8376144,151.251876,0.00000,15.300965,2314.299727
496,8376145,151.955755,0.00000,15.331567,2329.719901
497,8376146,161.211152,0.00000,15.362231,2476.562873
498,8376147,164.425335,0.00000,15.392955,2530.991781


In [1]:

tick_gap = 10

xticks = df.bin_id[::tick_gap]
xtick_lables = df.bin_price[::tick_gap]

fig, ax = plt.subplots()

ax.bar(list(df.bin_id), list(df.reserveY), label=tokenY_symbol, color='b', edgecolor="none")
ax.bar(list(df.bin_id), list(df.reserveX_in_Y), bottom=np.array(df.reserveY, dtype=float), label=tokenX_symbol, color='r', edgecolor="none")

ax.set_ylabel('TVL ($)')
ax.set_xlabel('Price ($)')
ax.set_title('LIquidity distribution per bin')
ax.legend()

ax.ticklabel_format(style='plain', useOffset=False)
fig.set_size_inches(18.5, 10.5, forward=True)
ax.set_xticks(xticks)
ax.set_xticklabels(np.round(xtick_lables, 4), rotation=90)
# plt.xticks(xticks, rotation=90)
ax.axvline(x=active_bin, color='red', linestyle='--', label='Active Bin')
ax.legend()
plt.show()

NameError: name 'df' is not defined

In [46]:
xticks[250]

8375899

In [83]:
df[(df['bin_price'] > 2.68)]["reserveX"].sum()

1300.1263747663477

In [34]:
xtick_lables

0       5.679747
50      6.276464
100     6.935873
150     7.664559
200     8.469802
250     9.378363
300    10.363660
350    11.452471
400    12.655674
450    13.985286
Name: bin_price, dtype: float64